<a href="https://colab.research.google.com/github/lauracguevaraa-tech/Base-de-Datos-Real/blob/main/Copia_de_Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =============================
# 1. CREAR BASE DE DATOS
# =============================
import sqlite3
import pandas as pd
import os

# Eliminar la base de datos si existe para evitar errores de integridad
if os.path.exists("chocolates.db"):
    os.remove("chocolates.db")

# Crear conexión
conn = sqlite3.connect("chocolates.db")
cursor = conn.cursor()

# =============================
# 2. CREAR TABLAS (10 procesos)
# =============================

# 1. Recepción, Selección y Tostado
cursor.execute("""
CREATE TABLE IF NOT EXISTS Recepcion_Tostado (
    ID_Lote TEXT PRIMARY KEY,
    Origen TEXT,
    Peso_kg REAL,
    Humedad REAL,
    Color_Corte TEXT,
    Tamano_Grano TEXT,
    Fecha_Recepcion TEXT,
    Inspector TEXT,
    Codigo_Trazabilidad TEXT,
    Temp_Tostado REAL,
    Duracion_min INTEGER,
    Peso_Tostado_kg REAL
)
""")

# 2. Descascarillado y Molienda
cursor.execute("""
CREATE TABLE IF NOT EXISTS Descascarillado_Molienda (
    ID_Molienda TEXT PRIMARY KEY,
    ID_Lote TEXT,
    Rendimiento REAL,
    Tamano_1ra_micras INTEGER,
    Tamano_2da_micras INTEGER,
    Temp_C REAL,
    Fecha TEXT,
    FOREIGN KEY (ID_Lote) REFERENCES Recepcion_Tostado(ID_Lote)
)
""")

# 3. Refinado y Conchado
cursor.execute("""
CREATE TABLE IF NOT EXISTS Refinado_Conchado (
    ID_Conchado TEXT PRIMARY KEY,
    ID_Molienda TEXT,
    Micras_Final INTEGER,
    Conchado_Horas INTEGER,
    Fase TEXT,
    Temp_C REAL,
    Fecha TEXT,
    FOREIGN KEY (ID_Molienda) REFERENCES Descascarillado_Molienda(ID_Molienda)
)
""")

# 4. Control - Verificación de Textura y Granulometría
cursor.execute("""
CREATE TABLE IF NOT EXISTS Control_Textura (
    ID_Control TEXT PRIMARY KEY,
    ID_Conchado TEXT,
    Particulas_25um REAL,
    Viscosidad_cP REAL,
    Fundicion_seg INTEGER,
    Sensorial TEXT,
    Tecnico TEXT,
    FOREIGN KEY (ID_Conchado) REFERENCES Refinado_Conchado(ID_Conchado)
)
""")

# 5. Temperado
cursor.execute("""
CREATE TABLE IF NOT EXISTS Temperado (
    ID_Temperado TEXT PRIMARY KEY,
    ID_Control TEXT,
    Tipo TEXT,
    T_Inicial REAL,
    T_Enfriado REAL,
    T_Final REAL,
    Fecha TEXT,
    FOREIGN KEY (ID_Control) REFERENCES Control_Textura(ID_Control)
)
""")

# 6. Moldeo
cursor.execute("""
CREATE TABLE IF NOT EXISTS Moldeo (
    ID_Moldeo TEXT PRIMARY KEY,
    ID_Temperado TEXT,
    Molde TEXT,
    Ciclos_Vibracion INTEGER,
    Estado TEXT,
    Fecha TEXT,
    FOREIGN KEY (ID_Temperado) REFERENCES Temperado(ID_Temperado)
)
""")

# 7. Enfriamiento
cursor.execute("""
CREATE TABLE IF NOT EXISTS Enfriamiento (
    ID_Enfriamiento TEXT PRIMARY KEY,
    ID_Moldeo TEXT,
    Temp_C REAL,
    HR REAL,
    Tiempo_min INTEGER,
    Fecha TEXT,
    FOREIGN KEY (ID_Moldeo) REFERENCES Moldeo(ID_Moldeo)
)
""")

# 8. Control - Inspección Visual
cursor.execute("""
CREATE TABLE IF NOT EXISTS Control_Acabado (
    ID_Acabado TEXT PRIMARY KEY,
    ID_Enfriamiento TEXT,
    Brillo TEXT,
    Defectos TEXT,
    Clasificacion TEXT,
    Inspector TEXT,
    FOREIGN KEY (ID_Enfriamiento) REFERENCES Enfriamiento(ID_Enfriamiento)
)
""")

# 9. Desmoldado y Decoración
cursor.execute("""
CREATE TABLE IF NOT EXISTS Desmoldado_Decoracion (
    ID_Decoracion TEXT PRIMARY KEY,
    ID_Acabado TEXT,
    Metodo TEXT,
    Decoracion TEXT,
    Responsable TEXT,
    FOREIGN KEY (ID_Acabado) REFERENCES Control_Acabado(ID_Acabado)
)
""")

# 10. Empaque
cursor.execute("""
CREATE TABLE IF NOT EXISTS Empaque (
    ID_Empaque TEXT PRIMARY KEY,
    ID_Decoracion TEXT,
    Empaque TEXT,
    Etiqueta TEXT,
    Fecha TEXT,
    Responsable TEXT,
    FOREIGN KEY (ID_Decoracion) REFERENCES Desmoldado_Decoracion(ID_Decoracion)
)
""")

# =============================
# 3. INSERTAR DATOS DE EJEMPLO
# =============================

# 1. Recepción y Tostado
cursor.executemany("""
INSERT INTO Recepcion_Tostado VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""", [
    ("L001", "Venezuela", 65, 7.2, "Púrpura", "Grande", "2025-09-20", "Juan R.", "VC-20250920-001", 142, 60, 25),
    ("L002", "Ecuador", 55, 6.5, "Púrpura", "Mediano", "2025-09-21", "María P.", "EC-20250921-002", 145, 72, 23),
    ("L003", "Madagascar", 70, 7.8, "Púrpura", "Pequeño", "2025-09-22", "Carlos S.", "MG-20250922-003", 140, 50, 24)
])

# 2. Descascarillado y Molienda
cursor.executemany("""
INSERT INTO Descascarillado_Molienda VALUES (?, ?, ?, ?, ?, ?, ?)
""", [
    ("M001", "L001", 83, 400, 250, 55, "2025-09-20"),
    ("M002", "L002", 85, 350, 220, 58, "2025-09-21"),
    ("M003", "L003", 80, 500, 300, 57, "2025-09-22")
])

# 3. Refinado y Conchado
cursor.executemany("""
INSERT INTO Refinado_Conchado VALUES (?, ?, ?, ?, ?, ?, ?)
""", [
    ("C001", "M001", 23, 48, "Húmedo", 63, "2025-09-21"),
    ("C002", "M002", 25, 60, "Seco", 68, "2025-09-22"),
    ("C003", "M003", 21, 72, "Húmedo", 65, "2025-09-23")
])

# 4. Control - Textura
cursor.executemany("""
INSERT INTO Control_Textura VALUES (?, ?, ?, ?, ?, ?, ?)
""", [
    ("CT001", "C001", 92, 3200, 20, "Cremoso", "Laura G."),
    ("CT002", "C002", 89, 2800, 22, "Adecuado", "Pedro H."),
    ("CT003", "C003", 94, 4100, 18, "Excelente", "Diana L.")
])

# 5. Temperado
cursor.executemany("""
INSERT INTO Temperado VALUES (?, ?, ?, ?, ?, ?, ?)
""", [
    ("TP001", "CT001", "Negro", 45, 27, 31, "2025-09-21"),
    ("TP002", "CT002", "Leche", 44, 26, 30, "2025-09-22"),
    ("TP003", "CT003", "Negro", 46, 27, 32, "2025-09-23")
])

# 6. Moldeo
cursor.executemany("""
INSERT INTO Moldeo VALUES (?, ?, ?, ?, ?, ?)
""", [
    ("MO001", "TP001", "Tableta 70%", 3, "Nivelado", "2025-09-21"),
    ("MO002", "TP002", "Bombón Leche", 3, "Nivelado", "2025-09-22"),
    ("MO003", "TP003", "Trufa Negra", 3, "Nivelado", "2025-09-23")
])

# 7. Enfriamiento
cursor.executemany("""
INSERT INTO Enfriamiento VALUES (?, ?, ?, ?, ?, ?)
""", [
    ("E001", "MO001", 17, 52, 45, "2025-09-21"),
    ("E002", "MO002", 16, 50, 60, "2025-09-22"),
    ("E003", "MO003", 18, 55, 30, "2025-09-23")
])

# 8. Control - Acabado
cursor.executemany("""
INSERT INTO Control_Acabado VALUES (?, ?, ?, ?, ?, ?)
""", [
    ("A001", "E001", "Alto", "Ninguno", "Primera", "Sofía M."),
    ("A002", "E002", "Medio", "Ligeras manchas", "Segunda", "Camilo V."),
    ("A003", "E003", "Alto", "Ninguno", "Primera", "Andrés T.")
])

# 9. Desmoldado y Decoración
cursor.executemany("""
INSERT INTO Desmoldado_Decoracion VALUES (?, ?, ?, ?, ?)
""", [
    ("D001", "A001", "Golpe seco", "Almendras", "Paula R."),
    ("D002", "A002", "Golpe seco", "Sin decoración", "Jorge S."),
    ("D003", "A003", "Golpe seco", "Transfer sheet dorado", "Felipe M.")
])

# 10. Empaque
cursor.executemany("""
INSERT INTO Empaque VALUES (?, ?, ?, ?, ?, ?)
""", [
    ("EP001", "D001", "Caja 100g", "Lote VC-20250920-001", "2025-09-21", "Ana C."),
    ("EP002", "D002", "Estuche 50g", "Lote EC-20250921-002", "2025-09-22", "Luis R."),
    ("EP003", "D003", "Caja 150g", "Lote MG-20250922-003", "2025-09-23", "Marta J.")
])

# Guardar cambios
conn.commit()

# =============================
# 4. FUNCIÓN PARA CONSULTAR
# =============================
def ver_tabla(nombre):
    df = pd.read_sql_query(f"SELECT * FROM {nombre}", conn)
    return df

# =============================
# 5. EJEMPLOS DE CONSULTA
# =============================
print("📦 Recepción y Tostado:")
display(ver_tabla("Recepcion_Tostado"))

print("🔥 Descascarillado y Molienda:")
display(ver_tabla("Descascarillado_Molienda"))

print("🍫 Refinado y Conchado:")
display(ver_tabla("Refinado_Conchado"))

print("✅ Control de Textura:")
display(ver_tabla("Control_Textura"))

print("🌡️ Temperado:")
display(ver_tabla("Temperado"))

print("🛠️ Moldeo:")
display(ver_tabla("Moldeo"))

print("❄️ Enfriamiento:")
display(ver_tabla("Enfriamiento"))

print("🔍 Control de Acabado:")
display(ver_tabla("Control_Acabado"))

print("🎨 Desmoldado y Decoración:")
display(ver_tabla("Desmoldado_Decoracion"))

print("📦 Empaque:")
display(ver_tabla("Empaque"))

📦 Recepción y Tostado:


,ID_Lote,Origen,Peso_kg,Humedad,Color_Corte,Tamano_Grano,Fecha_Recepcion,Inspector,Codigo_Trazabilidad,Temp_Tostado,Duracion_min,Peso_Tostado_kg
0,L001,Venezuela,65.0,7.2,Púrpura,Grande,2025-09-20,Juan R.,VC-20250920-001,142.0,60,25.0
1,L002,Ecuador,55.0,6.5,Púrpura,Mediano,2025-09-21,María P.,EC-20250921-002,145.0,72,23.0
2,L003,Madagascar,70.0,7.8,Púrpura,Pequeño,2025-09-22,Carlos S.,MG-20250922-003,140.0,50,24.0


🔥 Descascarillado y Molienda:


,ID_Molienda,ID_Lote,Rendimiento,Tamano_1ra_micras,Tamano_2da_micras,Temp_C,Fecha
0,M001,L001,83.0,400,250,55.0,2025-09-20
1,M002,L002,85.0,350,220,58.0,2025-09-21
2,M003,L003,80.0,500,300,57.0,2025-09-22


🍫 Refinado y Conchado:


...

,ID_Acabado,ID_Enfriamiento,Brillo,Defectos,Clasificacion,Inspector
0,A001,E001,Alto,Ninguno,Primera,Sofía M.
1,A002,E002,Medio,Ligeras manchas,Segunda,Camilo V.
2,A003,E003,Alto,Ninguno,Primera,Andrés T.


🎨 Desmoldado y Decoración:


,ID_Decoracion,ID_Acabado,Metodo,Decoracion,Responsable
0,D001,A001,Golpe seco,Almendras,Paula R.
1,D002,A002,Golpe seco,Sin decoración,Jorge S.
2,D003,A003,Golpe seco,Transfer sheet dorado,Felipe M.


📦 Empaque:


,ID_Empaque,ID_Decoracion,Empaque,Etiqueta,Fecha,Responsable
0,EP001,D001,Caja 100g,Lote VC-20250920-001,2025-09-21,Ana C.
1,EP002,D002,Estuche 50g,Lote EC-20250921-002,2025-09-22,Luis R.
2,EP003,D003,Caja 150g,Lote MG-20250922-003,2025-09-23,Marta J.
